In [3]:
import numpy as np
import sys
PYTHONPATH = r"C:\Users\12429"
sys.path.append(PYTHONPATH)
print(sys.path)
import MLMD

['c:\\Users\\12429\\MLMD', 'd:\\anaconda\\envs\\torch_cuda_11_3\\python39.zip', 'd:\\anaconda\\envs\\torch_cuda_11_3\\DLLs', 'd:\\anaconda\\envs\\torch_cuda_11_3\\lib', 'd:\\anaconda\\envs\\torch_cuda_11_3', '', 'C:\\Users\\12429\\AppData\\Roaming\\Python\\Python39\\site-packages', 'd:\\anaconda\\envs\\torch_cuda_11_3\\lib\\site-packages', 'd:\\anaconda\\envs\\torch_cuda_11_3\\lib\\site-packages\\win32', 'd:\\anaconda\\envs\\torch_cuda_11_3\\lib\\site-packages\\win32\\lib', 'd:\\anaconda\\envs\\torch_cuda_11_3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\12429', 'C:\\Users\\12429']


读取XDATCAR文件，步长为2fs

In [ ]:
LiF = MLMD.utils.io.loadfile(2)

制作测试集

In [ ]:
train_data = np.array([np.array(np.append(LiF["F"].position[i],LiF["Li"].position[i])) for i in range(len(LiF["F"].position))])
train_data

In [ ]:
import torch
import torch.nn as nn
import numpy as np

用前五帧的路径去预测第六帧

In [ ]:
fwd = 5




In [ ]:
# Create a MyDNN instance
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MyDNN().to(device)

num_epochs = 1000

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

loss_array = []   #draw picture

# Train the model
for epoch in range(num_epochs):
    # Get the input and target arrays
    
    input_array = torch.from_numpy(train_data[epoch:epoch+fwd].reshape(-1)).float().to(device)
    target_array = torch.from_numpy(train_data[epoch+fwd+1].reshape(-1)).float().to(device)

    # Forward pass
    output_array = model(input_array)

    # Compute the loss
    loss = criterion(output_array, target_array)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.9f}'.format(epoch+1, num_epochs, loss.item()))
        loss_array.append(loss.item())

In [ ]:
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

plt.figure(dpi=120)
plt.plot(loss_array)
plt.yscale("log")

用frame的前五帧预测下一帧

In [ ]:
# Get the input array for prediction
frame = 100
input_array = torch.from_numpy(train_data[frame-5:frame].reshape(-1)).float().to(device)

# Predict the output array
with torch.no_grad():
    output_array = model(input_array)

# Convert the output tensor to a numpy array
output_array = output_array.cpu().numpy()

In [ ]:
#print(output_array.reshape(-1,3))

In [ ]:
import ase
from ase.io import read ,write
from ase.build import sort
from ase.visualize import view

# 将某一帧写出

In [ ]:
with open('LiF.vasp', 'w') as f:
    f.write("vasp\n")
    f.write("1.0\n")
    for i in (LiF["F"].lattice):
        f.write(str(i)[1:-1]+"\n")
    for (i,j) in LiF.items():
        f.write(j.name + " ")
    f.write("\n")
    for (i,j) in LiF.items():
        f.write(str(j.number) + " ")
    f.write("\n")
    f.write("Direct\n")
    for i in output_array.reshape(-1,3):
        f.write(str(i)[1:-1]+"\n")

bulk = read("LiF.vasp")
view(bulk)

# 跑10000步写成xdatcar00(还没写完)

In [ ]:
running_test = train_data[:5]

In [ ]:
for i in range(10000):
    input_array = torch.from_numpy(running_test[-5:].reshape(-1)).float().to(device)
    with torch.no_grad():
        output_array = model(input_array)
    running_test.append(output_array.cpu().numpy())

In [ ]:
running_test